In [23]:
import torch
import os
import json
import utils
import sys
import model.net as net  # Ensure this matches your model definition
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay 

from scipy import stats
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
# matplotlib.use("TkAgg")  # Use a GUI-compatible backend
matplotlib.use('Agg')  # replace TkAgg as Agg

## Comparison Graph

In [26]:
# Setup paths
capstone_dir = os.path.join(os.path.expanduser("~"), "CAPSTONE-stockreturn")
model_dir = os.path.join(capstone_dir, "DeepAR", "experiments", "goog_base_model")
data_path = os.path.join(capstone_dir, "data", "stock", "goog_stock_wsenti.csv")

# Load the dataset
data = pd.read_csv(data_path, parse_dates=['Date'])
data['Date'] = pd.to_datetime(data['Date'])
data.set_index("Date", inplace=True)

# Parameters for prediction
forecast_start = "2025-02-18"  # date started to predict
end_date = "2025-02-24"
train_window = 30
num_covariates = 9
future_steps = 5  # predict number of days

# Add path for model imports
sys.path.append(os.path.join(capstone_dir, "DeepAR"))

# Initialize parameters
params = utils.Params(os.path.join(model_dir, "params.json"))
params.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the checkpoint files to process
checkpoint_files = [f"epoch_{i}.pth.tar" for i in range(15)] + ["best.pth.tar"]

# Loop through each checkpoint
for checkpoint_file in checkpoint_files:
    print(f"Processing checkpoint: {checkpoint_file}")
    
    # Extract epoch number or 'best' for filename
    if checkpoint_file == "best.pth.tar":
        epoch_label = "best"
    else:
        epoch_label = checkpoint_file.split("_")[1].split(".")[0]  # Extract number after "epoch_"
    
    # Load the model for this checkpoint
    model = net.Net(params).to(params.device)
    utils.load_checkpoint(os.path.join(model_dir, checkpoint_file), model)
    model.eval()
    
    # Get the training data
    last_30_days = data.loc[:forecast_start].iloc[-train_window:]
    last_30_days.fillna(method='ffill', inplace=True)
    price_data = last_30_days[['High', 'Low', 'Open', 'Close', 'Volume', 'Sentiment_Score', 'Daily Return']]
    # price_data = last_30_days[['High', 'Low', 'Open', 'Close', 'Volume', 'Daily Return']]
    
    # Generate covariates
    def gen_covariates(times, price_data, num_covariates):
        covariates = np.zeros((len(times), num_covariates))
        covariates[:, 0] = stats.zscore([t.weekday() for t in times])
        covariates[:, 1] = stats.zscore([t.month for t in times])
        covariates[:, 2] = stats.zscore(price_data['Close'].shift(5).values)
        covariates[:, 3] = stats.zscore(price_data['Volume'].shift(5).values)
        intraday_return = (price_data['Close'] - price_data['Open']) / price_data['Open']
        covariates[:, 4] = stats.zscore(intraday_return.shift(5).values)
        ma5 = price_data['Close'].rolling(window=5).mean()
        covariates[:, 5] = stats.zscore((price_data['Close'] - ma5).values)
        exp1 = price_data['Close'].ewm(span=12, adjust=False).mean()
        exp2 = price_data['Close'].ewm(span=26, adjust=False).mean()
        macd = exp1 - exp2
        covariates[:, 6] = stats.zscore(macd.shift(2).values)
        volatility = (price_data['High'] - price_data['Low']) / price_data['Close']
        covariates[:, 7] = stats.zscore(volatility.values)
        covariates[:, 8] = stats.zscore(price_data['Sentiment_Score'].shift(5).values)
        return np.nan_to_num(covariates)
    
    covariates = gen_covariates(last_30_days.index, price_data, num_covariates)
    
    # Prepare initial input tensor
    x_input = np.zeros((1, train_window, 1 + num_covariates), dtype='float32')
    x_input[0, 1:, 0] = last_30_days['Daily Return'].values[1:]
    x_input[0, :, 1:1 + num_covariates] = covariates[-train_window:, :]
    new_input_tensor = torch.tensor(x_input, dtype=torch.float32).permute(1, 0, 2).to(params.device)
    
    # Generate future trading days
    start_date = pd.to_datetime(forecast_start)
    future_trading_days = pd.date_range(start=start_date, periods=future_steps, freq=BDay())
    
    # Predict for trading days
    batch_size = new_input_tensor.shape[1]
    hidden = model.init_hidden(batch_size)
    cell = model.init_cell(batch_size)
    idx = torch.zeros(1, batch_size, dtype=torch.long, device=params.device)
    predictions = []
    
    for _ in range(future_steps):
        mu, sigma, hidden, cell = model(new_input_tensor[-1].unsqueeze(0), idx, hidden, cell)
        next_value = mu.cpu().detach().numpy().squeeze()
        predictions.append(next_value)
        new_input = np.roll(new_input_tensor.cpu().numpy(), shift=-1, axis=0)
        new_input[-1, 0, 0] = next_value
        new_input_tensor = torch.tensor(new_input, dtype=torch.float32).to(params.device)
    
    # Convert returns to stock prices
    last_price = last_30_days['Close'].iloc[-1]
    predicted_prices = [last_price]
    for ret in predictions:
        next_price = predicted_prices[-1] * (1 + ret)
        predicted_prices.append(next_price)
    predicted_prices = predicted_prices[1:]  # Remove the initial price
    
    # Get actual data for comparison
    actual_data = data.reset_index()
    actual_data = actual_data[(actual_data["Date"] >= forecast_start) & (actual_data["Date"] <= end_date)]
    
    # Create the combined plot
    plt.figure(figsize=(12, 6))
    
    # Plot predictions
    plt.plot(future_trading_days, predicted_prices, marker='o', linestyle='-', color='blue', label="Predicted Prices")
    
    # Plot actual prices
    plt.plot(actual_data["Date"], actual_data["Close"], marker='x', linestyle='--', color='green', label="Actual Prices")
    
    # Add vertical line at prediction start
    plt.axvline(x=pd.to_datetime(forecast_start), color='red', linestyle='--', label='Prediction Start')
    
    # Format the plot
    plt.title(f"GOOG Stock: Predicted vs Actual Prices (Checkpoint: {checkpoint_file})")
    plt.xlabel("Date")
    plt.ylabel("Stock Price")
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator())
    plt.xticks(rotation=45)
    
    # Calculate metrics
    actual_data_dates_str = [d.strftime('%Y-%m-%d') for d in actual_data["Date"]]
    future_days_str = [d.strftime('%Y-%m-%d') for d in future_trading_days]
    
    matching_actual = []
    matching_indices = []
    
    for i, pred_date_str in enumerate(future_days_str):
        if pred_date_str in actual_data_dates_str:
            idx = actual_data_dates_str.index(pred_date_str)
            matching_actual.append(actual_data["Close"].iloc[idx])
            matching_indices.append(i)
    
    matching_actual = np.array(matching_actual)
    predicted_prices_array = np.array(predicted_prices)
    
    # Basic metrics
    mae = np.mean(np.abs(predicted_prices_array[matching_indices] - matching_actual))
    mape = np.mean(np.abs((matching_actual - predicted_prices_array[matching_indices]) / matching_actual)) * 100
    rmse = np.sqrt(np.mean((predicted_prices_array[matching_indices] - matching_actual)**2))
    
    # Direction accuracy
    if len(matching_actual) > 1:  # Ensure we have at least 2 points for direction
        actual_direction = np.diff(matching_actual) > 0
        predicted_direction = np.diff(predicted_prices_array[matching_indices]) > 0
        direction_accuracy = np.mean(actual_direction == predicted_direction) * 100
    else:
        direction_accuracy = float('nan')
    
    # Short-term vs long-term accuracy
    if len(matching_actual) >= 2:
        short_term_mae = np.mean(np.abs(predicted_prices_array[matching_indices[:2]] - matching_actual[:2]))
    else:
        short_term_mae = float('nan')
        
    if len(matching_actual) >= 5:
        long_term_mae = np.mean(np.abs(predicted_prices_array[matching_indices[3:]] - matching_actual[3:]))
    else:
        long_term_mae = float('nan')
    
    # Add a text box with metrics to the plot
    metrics_text = (
        f"Metrics:\n"
        f"MAE: ${mae:.2f}\n"
        f"MAPE: {mape:.2f}%\n"
        f"RMSE: ${rmse:.2f}\n"
        f"Direction Accuracy: {direction_accuracy:.1f}%\n"
        f"Short-term MAE (1-2d): ${short_term_mae:.2f}\n"
        f"Long-term MAE (4-5d): ${long_term_mae:.2f}"
    )
    
    # Position the text box in the upper right corner with some padding
    plt.annotate(
        metrics_text,
        xy=(0.97, 0.97),
        xycoords='axes fraction',
        fontsize=9,
        ha='right',
        va='top',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8)
    )
    
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    
    # Save the figure with a unique name based on the checkpoint
    figures_dir = os.path.join(model_dir, "figures", "goog_compare_epochs")
    os.makedirs(figures_dir, exist_ok=True)
    plot_path = os.path.join(figures_dir, f"goog_prediction_vs_actual_with_metrics_{epoch_label}.png")
    plt.savefig(plot_path)
    plt.close()
    
    print(f"Saved plot for checkpoint {checkpoint_file} to {plot_path}")
    
print("Completed processing all checkpoints!")

Processing checkpoint: epoch_0.pth.tar


/home/yic075/CAPSTONE-stockreturn/DeepAR/utils.py:148: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint, map_location='cpu')
/tmp/ipykernel_1

Saved plot for checkpoint epoch_0.pth.tar to /home/yic075/CAPSTONE-stockreturn/DeepAR/experiments/goog_base_model/figures/goog_compare_epochs/goog_prediction_vs_actual_with_metrics_0.png
Processing checkpoint: epoch_1.pth.tar


/home/yic075/CAPSTONE-stockreturn/DeepAR/utils.py:148: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint, map_location='cpu')
/tmp/ipykernel_1

Saved plot for checkpoint epoch_1.pth.tar to /home/yic075/CAPSTONE-stockreturn/DeepAR/experiments/goog_base_model/figures/goog_compare_epochs/goog_prediction_vs_actual_with_metrics_1.png
Processing checkpoint: epoch_2.pth.tar


...

Saved plot for checkpoint epoch_13.pth.tar to /home/yic075/CAPSTONE-stockreturn/DeepAR/experiments/goog_base_model/figures/goog_compare_epochs/goog_prediction_vs_actual_with_metrics_13.png
Processing checkpoint: epoch_14.pth.tar


/home/yic075/CAPSTONE-stockreturn/DeepAR/utils.py:148: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint, map_location='cpu')
/tmp/ipykernel_1

Saved plot for checkpoint epoch_14.pth.tar to /home/yic075/CAPSTONE-stockreturn/DeepAR/experiments/goog_base_model/figures/goog_compare_epochs/goog_prediction_vs_actual_with_metrics_14.png
Processing checkpoint: best.pth.tar


/home/yic075/CAPSTONE-stockreturn/DeepAR/utils.py:148: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint, map_location='cpu')
/tmp/ipykernel_1

Saved plot for checkpoint best.pth.tar to /home/yic075/CAPSTONE-stockreturn/DeepAR/experiments/goog_base_model/figures/goog_compare_epochs/goog_prediction_vs_actual_with_metrics_best.png
Completed processing all checkpoints!
